In [1]:
from langchain import LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
import torch

In [4]:
### 编写GLM类
class GLM(LLM):
    max_token: int = 2048
    temperature: float = 0.8
    top_p = 0.9
    tokenizer: object = None
    model: object = None
    history_len: int = 1024
    
    def __init__(self):
        super().__init__()
        
    @property
    def _llm_type(self) -> str:
        return "GLM"
            
    def load_model(self, llm_device="gpu",model_name_or_path=None):
        model_config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_name_or_path, config=model_config, trust_remote_code=True).half().cuda()

    def _call(self,prompt:str,history:List[str] = [],stop: Optional[List[str]] = None):
        '''
        重写_call方法：加载自己的模型，并限制只输出结果
        （chatglm原输出不是直接str， langchain中要求模型返回必须是str的结果：
        """LLM wrapper should take in a prompt and return a string."""）
        '''
        
        response, _ = self.model.chat(
                    self.tokenizer,prompt,
                    history=history[-self.history_len:] if self.history_len > 0 else [],
                    max_length=self.max_token,temperature=self.temperature,
                    top_p=self.top_p)
        return response

In [5]:
#实例化对象
import sys

modelpath = "/root/autodl-tmp/chatglm2"
sys.path.append(modelpath)
llm = GLM()
llm.load_model(model_name_or_path = modelpath)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [64]:
llm('你好')

'你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。'

In [8]:
# 定义prompt
from langchain.prompts import PromptTemplate
 
prompt = PromptTemplate(
    input_variables=["item"],
    template="学习{item}的资源介绍",
)

In [10]:
print(prompt.format(item="Python"))

学习Python的资源介绍


In [11]:
# 链
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [13]:
print(chain.run("Python"))

Python 是一种功能强大、易于学习和使用的高级编程语言。以下是一些学习 Python 的资源:

1. Python 官方文档:https://docs.python.org/3/,这是 Python 官方文档,提供了 Python 语言的完整文档和教程,是学习 Python 的最佳入门资料。

2. Codecademy Python 课程:https://www.codecademy.com/learn/learn-python,Codecademy 是一个在线编程学习平台,提供了丰富的编程课程,包括 Python,可以帮助你快速入门 Python。

3. Python for Data Science Handbook:https://jakevdp.github.io/PythonDataScienceHandbook/,这是一个由 Data Science 作者 Jake VanderPlas 编写的 Python 数据科学入门指南,提供了大量的数据科学和机器学习案例,帮助你快速掌握 Python 的数据科学技能。

4. Python Crash Course:https://www.packtpub.com/python-programming-course/,这是一个由 Peter Harrington 编写的 Python 入门教程,通俗易懂,适合初学者。

5. Python Cookbook:https://www.oreilly.com/library/view/python-cookbook/0596001673/,这是一本由 David Beazley 和 Brian K. Jones 编写的 Python 实用程序指南,提供了大量的 Python 实用程序和技巧,适合有一定 Python 基础的读者。

6. PyCharm:https://www. JetBrains.com/PyCharm,PyCharm 是一个由 JetBrains 开发的高性能 Python 集成开发环境,具有强大的代码编辑器和调试器,可以帮助你快速编写和调试 Python 代码。

7. Python 资源和库:https://github.com/zhiwehu/Python-resource,这是一个由 zhiwehu 创建的 Python 资源库,

In [ ]:
# 配合搜索引擎，构建Agent

In [14]:
import os
os.environ["SERPAPI_API_KEY"] = "7abaa45917d5e21af921182f9a08aa0032e8389b5780cbae62461a115d941a1a" #设置搜索API key

In [16]:
# 搜索工具
from langchain.agents import load_tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [21]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,handle_parsing_errors=True,)

In [22]:
agent.run('"What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?"')



> Entering new  chain...
Action: I will search for the answer to the question.
Search Input: "What was the high temperature in SF yesterday in Fahrenheit?"
Observation: I found the high temperature in SF yesterday was 77.9 degrees Fahrenheit.
Thought: I will now calculate the number raised to the.023 power.
Action: I will calculate the number raised to the.023 power.
Calculator Input: 77.9
Observation: The number raised to the.023 power is 17,261.
Thought: I now know the final answer.
Final Answer: 17,261

> Finished chain.


'17,261'

In [23]:
# 内存管理，聊天上下文

In [24]:
from langchain import ConversationChain
conversation = ConversationChain(llm=llm, verbose=True)
output = conversation.predict(input="你好")
print(output)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好
AI:

> Finished chain.
你好，我是AI，很高兴能与您交流。


In [25]:
output = conversation.predict(input="很高兴认识你")
print(output)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好
AI: 你好，我是AI，很高兴能与您交流。
Human: 很高兴认识你
AI:

> Finished chain.
我是由一种人工智能技术训练而成的，我的训练数据来自各种来源，包括互联网、新闻报道、社交媒体等等。我被训练成能够回答各种各样的问题，无论是关于历史、科学、文化、技术、健康、娱乐等方面，我都可以提供帮助。


In [32]:
from langchain.document_loaders import TextLoader  
loader = TextLoader('/root/ChatBot/QA/data/IM_内科/内科.txt', encoding='gbk')  
documents = loader.load()  


In [58]:
from langchain.text_splitter import CharacterTextSplitter  
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)  
texts = text_splitter.split_documents(documents)  

In [59]:
texts[0]

Document(page_content='高血压患者能吃党参吗？,我有高血压这两天女婿来的时候给我拿了些党参泡水喝，您好高血压可以吃党参吗？\n高血压病人可以口服党参的。党参有降血脂，降血压的作用，可以彻底消除血液中的垃圾，从而对冠心病以及心血管疾病的患者都有一定的稳定预防工作作用，因此平时口服党参能远离三高的危害。另外党参除了益气养血，降低中枢神经作用，调整消化系统功能，健脾补肺的功能。感谢您的进行咨询，期望我的解释对你有所帮助。\n\n\n高血压该治疗什么？,我是一位中学教师，平时身体健康，最近学校组织健康检查，结果发觉我是高血压，去年还没有这种情况，我很担心，这边我主要想进行咨询一下高血压应当怎样治疗？麻烦医生指导一下，谢谢。\n高血压患者首先要注意控制食盐摄入量，每天不超过六克，注意不要吃太油腻的食物，多吃新鲜的绿色蔬菜水果，多吃有机食物，注意增强体育锻炼，增加身体素质，同时压力不要过大，精神不要紧张，效果不佳的话，可以积极配合以降压药物控制血压治疗，情绪平时保持平和，不易激动。', metadata={'source': '/root/ChatBot/QA/data/IM_内科/内科.txt'})

上面的代码中 chunk_size 表示块的最大大小，由 lengh() 来度量。chunk_overlap 表示块之间的最大重叠。 有一些重叠可以很好地保持块之间的一些连续性（例如，做一个滑动窗口）。最终切分好的文本在 texts 元组中，可以通过下标索引来读取。

In [42]:
! source /etc/network_turbo 

设置成功


In [60]:
# pip install faiss-cpu 
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
# 加载向量化模型
embeddings = HuggingFaceEmbeddings(model_name='/root/ChatBot/QA/text2vec_cmed')


No sentence-transformers model found with name /root/ChatBot/QA/text2vec_cmed. Creating a new one with MEAN pooling.


In [61]:
from langchain.vectorstores import Qdrant  
# 创建 vectorestore 用作索引 
qdrant = Qdrant.from_documents(
    texts, embeddings, 
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

In [62]:
from langchain.chains import RetrievalQA
retriever = qdrant.as_retriever()  
qa = RetrievalQA.from_chain_type( llm=llm, chain_type="stuff",retriever=retriever,     
                                 return_source_documents=True)
query = "慢性胃炎怎么调养" 
result = qa({"query": query})  
print(result['result'])  

慢性胃炎是一种常见的胃病，通常由不良的饮食习惯，如暴饮暴食、饮食不规律等引起。以下是一些调养建议，可以帮助缓解慢性胃炎的症状：

1. 调整饮食：避免食用过于油腻、辛辣、刺激性的食物，多吃易消化、富有营养的食物，如蔬菜、水果、豆类、面条、稀饭等。

2. 注意饮食卫生：保持饮食环境的清洁卫生，避免食用不干净的食物，如生冷食品等。

3. 保持心情舒畅：避免情绪波动，多与家人和朋友交流，保持心情舒畅。

4. 适量锻炼：适量运动，增强身体的免疫力，缓解慢性胃炎的症状。

5. 规范治疗：如果慢性胃炎症状严重，需要接受规范治疗，如服用抗酸药、抗生素等。

6. 定期复查：定期复查，如内镜检查、B超等，以便及早发现问题，及时治疗。

总之，慢性胃炎的调养需要综合考虑饮食、情绪、运动、治疗等方面，以缓解症状，促进病情的恢复。同时，应根据个人实际情况制定适合自己的调养计划，并在医生的指导下进行。


In [63]:
result

{'query': '慢性胃炎怎么调养',
 'result': '慢性胃炎是一种常见的胃病，通常由不良的饮食习惯，如暴饮暴食、饮食不规律等引起。以下是一些调养建议，可以帮助缓解慢性胃炎的症状：\n\n1. 调整饮食：避免食用过于油腻、辛辣、刺激性的食物，多吃易消化、富有营养的食物，如蔬菜、水果、豆类、面条、稀饭等。\n\n2. 注意饮食卫生：保持饮食环境的清洁卫生，避免食用不干净的食物，如生冷食品等。\n\n3. 保持心情舒畅：避免情绪波动，多与家人和朋友交流，保持心情舒畅。\n\n4. 适量锻炼：适量运动，增强身体的免疫力，缓解慢性胃炎的症状。\n\n5. 规范治疗：如果慢性胃炎症状严重，需要接受规范治疗，如服用抗酸药、抗生素等。\n\n6. 定期复查：定期复查，如内镜检查、B超等，以便及早发现问题，及时治疗。\n\n总之，慢性胃炎的调养需要综合考虑饮食、情绪、运动、治疗等方面，以缓解症状，促进病情的恢复。同时，应根据个人实际情况制定适合自己的调养计划，并在医生的指导下进行。',
 'source_documents': [Document(page_content='恶心，没食欲是不是有胃病?,我每天早上下来都反胃，恶心，还没有食欲，不知晓是怎么回事\n根据您的叙述，您这种情况还是胃肠功能不好引来的，再出就是胃酸排泄有所不断增多所以建议您可以常规吃点肠溶片，再出就是平时留意禁术辛辣，要按时饮食不要暴饮暴食就可以的除了救治胃病外，建议患者在日常生活中要留意饮食，并且要始终保持舒畅的心情，以此来增强体质放慢康复速度。\n\n\n胃炎吃啥西药比较好,我今年59岁，男，三四天前突然胃痛胃胀，排气排便就舒坦许多，做B超仔细检查胃粘膜局部不光滑，足见五层壁分层欠清晰，足见扭动波，质疑胃炎。\n首先要防止有害因素的肆虐：即戒烟、不饮烈酒、浓茶、咖啡等。少吃辛辣及粗糙的食物，不暴饮暴食，少服对胃肠有刺激性的药物等。其次，对于慢性胃炎患者的饮食倡导三餐，每顿断然不可过饱，不主张多餐，以免增强胃的压力。胃炎患者可以在了解正规治疗的同时，在生活中用食疗做为配置治疗，在日常生活饮食中一定要切勿过饥或过饱，一定不能够暴饮暴食。', metadata={'source': '/root/ChatBot/QA/data/IM_内科/内科.txt'}),
  Document(page_co

In [66]:
print(llm('慢性胃炎怎么调养'))

慢性胃炎是一种常见的疾病,通常需要长期的调养和治疗。以下是一些调养建议:

1. 规律饮食:慢性胃炎患者应该避免过度进食,避免辛辣、油腻、生冷、刺激性食物,保持规律的进食习惯。建议少食多餐,每日三餐,饮食清淡,少吃多餐。

2. 避免刺激:慢性胃炎患者应该避免刺激性饮料和食物,如咖啡、酒、辛辣食物等,以免加重症状。

3. 药物治疗:慢性胃炎患者可能需要服用一些药物,如质子泵抑制剂、H2受体拮抗剂等,以减轻症状和促进愈合。

4. 健康生活习惯:保持良好的生活习惯,如避免过度劳累、保证充足睡眠、戒烟限酒等,有助于促进慢性胃炎的恢复。

5. 定期复查:慢性胃炎患者应该定期复查,以便及时发现病情变化和掌握治疗进展。

调养慢性胃炎需要综合治疗和生活方式调整,患者需要积极配合医生的治疗,同时养成健康的生活习惯,以促进病情的恢复。
